This notebook contains the code for the meta-analysis of healthy lung data for ACE2, TMPRSS2, and CTSL. It contains the pseudo-bulk analysis for the simple model without interaction terms that was run on the patient-level data. This script contains the code that was run on the full data and does not test for smoking associations.

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import patches
import seaborn as sns
import batchglm
import diffxpy.api as de
import patsy as pat
from statsmodels.stats.multitest import multipletests
import logging, warnings
import statsmodels.api as sm

/home/icb/malte.luecken/anaconda3/envs/cov19_py/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()
de.__version__

logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 35)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==1.0.1 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


'v0.7.3'

In [3]:
#User inputs
folder = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study'

adata_diffxpy = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study/COVID19_lung_atlas_revision_v3.h5ad'

output_folder = 'diffxpy_out/'

de_output_base = 'COVID19_lung_atlas_revision_v3_lung_cov19_poissonglm_pseudo_nUMIoffset_noInts'

# Read the data

In [4]:
adata = sc.read(adata_diffxpy)

In [5]:
adata

AnnData object with n_obs × n_vars = 1320896 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [6]:
adata.obs.age = adata.obs.age.astype(float)

In [7]:
adata.obs.dtypes

age                         float64
anatomical_region          category
donor                      category
last_author/PI             category
lung_vs_nasal              category
notes                      category
original_celltype_ann      category
sample                     category
sex                        category
smoking                    category
total_counts                float64
smoked_boolean             category
last_author_sample_name    category
ann_level_1                category
ann_level_2                category
ann_level_3                category
ann_level_4                category
ann_level_5                category
ann_highest_res               int64
ann_new                        bool
dtype: object

In [8]:
adata.obs['dataset'] = adata.obs['last_author/PI']

In [9]:
adata.obs.dataset.value_counts()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Spence                      78401
Barbry/Leroy                76981
Krasnow/Quake               60993
Meyer                       57020
Rawlins                     53409
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Misharin                    28329
Xavier/Regev                25552
Spira/Campbell              24455
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Nawijn                      18197
Teichmann                   12971
Kropski/Banovich_dnar        8359
Shalek                       7603
Linnarsson                   4640
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      2207
Schultze/Falk                1965
Beane         

# Filter the data

Keep only datsets with:
- more than 1 donor
- non-fetal
- lung

In [10]:
# Remove fetal datasets
dats_to_remove = set(['Rawlins', 'Spence', 'Linnarsson'])

In [11]:
dat = adata.obs.groupby(['donor']).agg({'sex':'first', 'age':'first', 'dataset':'first'})

# Single donor filter
don_tab = dat['dataset'].value_counts()
dats_to_remove.update(set(don_tab.index[don_tab == 1]))

In [12]:
dats_to_remove = list(dats_to_remove)
dats_to_remove

['Spence', 'Linnarsson', 'Misharin', 'Rawlins', 'Schultze/Falk']

In [13]:
adata = adata[~adata.obs.dataset.isin(dats_to_remove)].copy()

In [14]:
adata.obs.lung_vs_nasal.value_counts()

lung     1096604
nasal      57548
Name: lung_vs_nasal, dtype: int64

In [15]:
# Filter for only lung data
adata = adata[adata.obs.lung_vs_nasal.isin(['lung']),].copy()

In [16]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset'

In [17]:
# Rename smoking status covariate
adata.obs['smoking_status'] = adata.obs.smoked_boolean

In [18]:
adata.obs.dataset.value_counts()
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Krasnow/Quake               60993
Barbry/Leroy                58790
Meyer                       57020
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Xavier/Regev                25552
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Teichmann                   12971
Nawijn                      11110
Kropski/Banovich_dnar        8359
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      1995
Beane                         886
Beane/Campbell                780
Name: dataset, dtype: int64

309

185

# Check the data

In [19]:
np.mean(adata.X.astype(int) != adata.X)

0.0

In [20]:
# Check if any non-integer data in a particular dataset
for dat in adata.obs.dataset.unique():
    val = np.mean(adata[adata.obs.dataset.isin([dat]),:].X.astype(int) != adata[adata.obs.dataset.isin([dat]),:].X)
    if val != 0:
        print(f'dataset= {dat}; value= {val}')
        adata[adata.obs.dataset.isin([dat]),:].X[:20,:20].A

All counts are integers

In [21]:
adata.obs.age.value_counts()
adata.obs.sex.value_counts()

57.00    81833
66.00    69371
42.00    66596
59.00    59955
18.00    59849
46.00    56291
64.00    46130
35.00    45869
0.25     29908
67.50    28201
55.50    27176
20.00    25362
51.00    24766
72.50    24366
30.00    24095
58.00    23723
29.00    19248
0.00     19065
32.00    19009
3.00     18387
42.50    16906
45.50    16351
56.00    16011
23.00    15181
41.00    14943
49.00    14225
57.50    13840
55.00    13733
68.00    11852
65.00    11404
75.00    11243
45.00    10939
47.00    10278
52.50    10133
21.00     9430
67.00     9278
27.00     8693
22.00     7980
63.00     7600
31.00     7085
33.00     6906
44.00     6681
62.00     6228
38.00     5629
73.00     5251
24.00     4433
60.00     4231
62.50     4073
26.00     4066
61.00     3922
50.00     3831
32.50     3772
52.00     3765
80.00     3261
10.00     2694
40.00     2647
17.00     2552
79.00     2550
54.00     2507
76.00     2447
36.00     2218
53.00     1592
37.00     1240
48.00     1153
81.00     1145
25.00      823
70.00     

female    583926
male      512678
Name: sex, dtype: int64

# Fit models and perform DE

In [22]:
cluster_key = 'ann_level_2'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Myeloid',
 'Airway epithelium',
 'Alveolar epithelium',
 'Lymphoid',
 'Fibroblast lineage',
 'Blood vessels',
 'Submucosal Gland',
 'Smooth Muscle',
 'Lymphatics',
 'Mesothelium',
 'Endothelial-like',
 'Granulocytes']

Calculate DE genes per cluster.

In [23]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status'

# Generate pseudobulks

In [24]:
for gene in adata.var_names:
    adata.obs[gene] = adata[:,gene].X.A

In [25]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status', 'ACE2', 'TMPRSS2', 'CTSL'

In [26]:
dat_pseudo = adata.obs.groupby(['donor', 'ann_level_2']).agg({'ACE2':'mean', 'TMPRSS2':'mean', 'CTSL':'mean', 'total_counts':'mean', 'age':'first', 'smoking_status':'first', 'sex':'first', 'dataset':'first'}).dropna().reset_index(level=[0,1])
adata_pseudo = sc.AnnData(dat_pseudo[['ACE2', 'TMPRSS2', 'CTSL']], obs=dat_pseudo.drop(columns=['ACE2', 'TMPRSS2', 'CTSL']))

Transforming to str index.


In [27]:
adata_pseudo.obs.head()

,donor,ann_level_2,total_counts,age,smoking_status,sex,dataset
0,1A1DFBng,1_Endothelial,6307.156863,47.0,True,female,Eils/Conrad/Kreuter
1,1A1DFBng,Airway epithelium,4948.705263,47.0,True,female,Eils/Conrad/Kreuter
2,1A1DFBng,Alveolar epithelium,4268.209154,47.0,True,female,Eils/Conrad/Kreuter
3,1A1DFBng,Fibroblast lineage,5888.260870,47.0,True,female,Eils/Conrad/Kreuter
4,1A1DFBng,Lymphatics,5503.117647,47.0,True,female,Eils/Conrad/Kreuter


In [28]:
adata_pseudo.obs['total_counts_scaled'] = adata_pseudo.obs['total_counts']/adata_pseudo.obs['total_counts'].mean()

In [29]:
formula = "1 + sex + age + dataset"
tested_coef = ["sex[T.male]", "age"]
dmat = de.utils.design_matrix(
    data=adata_pseudo,
    formula="~" + formula,
    as_numeric=["age"],
    return_type="patsy"
)
dmat[1]

['Intercept',
 'sex[T.male]',
 'dataset[T.Beane]',
 'dataset[T.Beane/Campbell]',
 'dataset[T.Eils/Conrad/Kreuter]',
 'dataset[T.Kaminski]',
 'dataset[T.Koenigshoff]',
 'dataset[T.Krasnow/Quake]',
 'dataset[T.Kropski/Banovich_dnar]',
 'dataset[T.Kropski/Banovich_vand]',
 'dataset[T.Lafyatis/Rojas]',
 'dataset[T.Mazzilli/Campbell/Beane]',
 'dataset[T.Meyer]',
 'dataset[T.Meyer_b]',
 'dataset[T.Misharin/Budinger]',
 'dataset[T.Nawijn]',
 'dataset[T.Regev]',
 'dataset[T.Regev/Rajagopal]',
 'dataset[T.Schiller]',
 'dataset[T.Schultze]',
 'dataset[T.Seibold]',
 'dataset[T.Teichmann]',
 'dataset[T.Whitsett/Xu_10X]',
 'dataset[T.Whitsett/Xu_dropSeq]',
 'dataset[T.Xavier/Regev]',
 'age']

## Poisson GLM

In [30]:
# Poisson GLM loop
de_results_lvl2_glm = dict()

# Test over clusters
for clust in clusters:
    adata_pseudo_tmp = adata_pseudo[adata_pseudo.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_pseudo_tmp.obs['smoking_status'].value_counts())
    print(adata_pseudo_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_pseudo_tmp, min_cells=4)
    if adata_pseudo_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_pseudo_tmp.obs.sex.value_counts())==1:
        print(f'{clust} only has 1 type of male/female sample.')
        continue
        
    print(f'Testing {adata_pseudo_tmp.n_vars} genes...')
    print(f'Testing in {adata_pseudo_tmp.n_obs} donors...')    
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_pseudo_tmp, #[idx_train],
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_pseudo_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_pseudo_tmp.X[:, i], #[idx_train, :], 
            exog=dmat[0],
            offset=np.log(adata_pseudo_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Test over coefs
        for coef in tested_coef:
            de_results_temp = pois_results.wald_test(
                [x for i, x in enumerate(pois_model.exog_names) if dmat[1][i] in [coef]]
            )

            # Output the results nicely
            de_results_temp = pd.DataFrame({
                "gene": gene,
                "cell_identity": clust,
                "covariate": coef,
                "coef": pois_results.params[[y == coef for y in dmat[1]]],
                "coef_sd": pois_results.bse[[y == coef for y in dmat[1]]],                 
                "pval": de_results_temp.pvalue
            }, index= [clust+"_"+gene+"_"+coef])

            de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl2_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl2_glm = pd.concat([de_results_lvl2_glm[i] for i in de_results_lvl2_glm.keys()], ignore_index=True)

In cluster Myeloid:
False    66
True     55
nan      21
Name: smoking_status, dtype: int64
male      76
female    66
Name: sex, dtype: int64
Testing 3 genes...
Testing in 142 donors...

In cluster Airway epithelium:
False    90
True     71
nan      20
Name: smoking_status, dtype: int64
male      97
female    84
Name: sex, dtype: int64
Testing 3 genes...
Testing in 181 donors...

In cluster Alveolar epithelium:
False    67
True     39
nan      14
Name: smoking_status, dtype: int64
male      64
female    56
Name: sex, dtype: int64
Testing 3 genes...
Testing in 120 donors...

In cluster Lymphoid:
False    72
True     62
nan      21
Name: smoking_status, dtype: int64
male      82
female    73
Name: sex, dtype: int64
Testing 3 genes...
Testing in 155 donors...

In cluster Fibroblast lineage:
False    54
True     40
nan      14
Name: smoking_status, dtype: int64
male      59
female    49
Name: sex, dtype: int64
Testing 3 genes...
Testing in 108 donors...

In cluster Blood vessels:
False    4

## Inspect some results

In [31]:
de_results_lvl2_glm.keys()

dict_keys(['Myeloid', 'Airway epithelium', 'Alveolar epithelium', 'Lymphoid', 'Fibroblast lineage', 'Blood vessels', 'Submucosal Gland', 'Smooth Muscle', 'Lymphatics', 'Mesothelium', 'Granulocytes'])

In [32]:
full_res_lvl2_glm

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],0.293275,6.455872,0.963766,0.963766
1,ACE2,Myeloid,age,-0.018336,0.231032,0.936741,0.963766
2,TMPRSS2,Myeloid,sex[T.male],0.772738,1.236718,0.532083,0.963766
3,TMPRSS2,Myeloid,age,0.003776,0.043545,0.930897,0.963766
4,CTSL,Myeloid,sex[T.male],-0.032409,0.108447,0.765060,0.963766
5,CTSL,Myeloid,age,0.003231,0.003391,0.340721,0.963766
6,ACE2,Airway epithelium,sex[T.male],0.030864,1.109078,0.977799,0.977799
7,ACE2,Airway epithelium,age,-0.005220,0.042812,0.902952,0.977799
8,TMPRSS2,Airway epithelium,sex[T.male],0.226545,0.286941,0.429810,0.977799
9,TMPRSS2,Airway epithelium,age,-0.004391,0.009659,0.649398,0.977799


In [33]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],0.293275,6.455872,0.963766,0.963766
1,ACE2,Myeloid,age,-0.018336,0.231032,0.936741,0.963766
6,ACE2,Airway epithelium,sex[T.male],0.030864,1.109078,0.977799,0.977799
7,ACE2,Airway epithelium,age,-0.005220,0.042812,0.902952,0.977799
12,ACE2,Alveolar epithelium,sex[T.male],0.200359,2.038741,0.921713,0.921713
13,ACE2,Alveolar epithelium,age,0.011537,0.068778,0.866783,0.921713
18,ACE2,Lymphoid,sex[T.male],-0.237169,1.640305,0.885036,0.885036
19,ACE2,Lymphoid,age,-0.108399,0.134471,0.420174,0.825247
24,ACE2,Fibroblast lineage,sex[T.male],0.124655,3.878615,0.974361,0.974361
25,ACE2,Fibroblast lineage,age,-0.008201,0.123681,0.947133,0.974361


In [34]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
2,TMPRSS2,Myeloid,sex[T.male],0.772738,1.236718,0.532083,0.963766
3,TMPRSS2,Myeloid,age,0.003776,0.043545,0.930897,0.963766
8,TMPRSS2,Airway epithelium,sex[T.male],0.226545,0.286941,0.429810,0.977799
9,TMPRSS2,Airway epithelium,age,-0.004391,0.009659,0.649398,0.977799
14,TMPRSS2,Alveolar epithelium,sex[T.male],0.199849,0.231938,0.388882,0.921713
15,TMPRSS2,Alveolar epithelium,age,0.008436,0.007439,0.256759,0.921713
20,TMPRSS2,Lymphoid,sex[T.male],0.346462,0.743801,0.641359,0.825247
21,TMPRSS2,Lymphoid,age,-0.010952,0.027246,0.687706,0.825247
26,TMPRSS2,Fibroblast lineage,sex[T.male],0.376760,0.855022,0.659471,0.974361
27,TMPRSS2,Fibroblast lineage,age,0.005276,0.029305,0.857134,0.974361


# Level 3 annotation

In [35]:
cluster_key = 'ann_level_3'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') or ct.startswith('2') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Macrophages',
 'AT2',
 'Basal',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'AT1',
 'Submucosal Secretory',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Myofibroblasts',
 'MDC',
 'Airway smooth muscle']

In [36]:
adata_sub = adata[adata.obs.ann_level_3.isin(clusters),:]

adata_sub
adata_sub.obs.donor.nunique()
adata_sub.obs['sample'].nunique()

View of AnnData object with n_obs × n_vars = 895743 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status', 'ACE2', 'TMPRSS2', 'CTSL'

185

309

## Generate pseudobulk

In [37]:
for gene in adata_sub.var_names:
    adata_sub.obs[gene] = adata_sub[:,gene].X.A

Trying to set attribute `.obs` of view, copying.


In [38]:
dat_pseudo_sub = adata_sub.obs.groupby(['donor', 'ann_level_3']).agg({'ACE2':'mean', 'TMPRSS2':'mean', 'CTSL':'mean', 'total_counts':'mean', 'age':'first', 'smoking_status':'first', 'sex':'first', 'dataset':'first'}).dropna().reset_index(level=[0,1])
adata_pseudo_sub = sc.AnnData(dat_pseudo_sub[['ACE2', 'TMPRSS2', 'CTSL']], obs=dat_pseudo_sub.drop(columns=['ACE2', 'TMPRSS2', 'CTSL']))

Transforming to str index.


In [39]:
adata_pseudo_sub.obs.head()

,donor,ann_level_3,total_counts,age,smoking_status,sex,dataset
0,1A1DFBng,AT1,3709.913265,47.0,True,female,Eils/Conrad/Kreuter
1,1A1DFBng,AT2,4453.519898,47.0,True,female,Eils/Conrad/Kreuter
2,1A1DFBng,Lymphatic EC,5503.117647,47.0,True,female,Eils/Conrad/Kreuter
3,1A1DFBng,Monocytes,3339.083948,47.0,True,female,Eils/Conrad/Kreuter
4,1A1DFBng,Multiciliated lineage,6000.121212,47.0,True,female,Eils/Conrad/Kreuter


In [40]:
adata_pseudo_sub.obs['total_counts_scaled'] = adata_pseudo_sub.obs['total_counts']/adata_pseudo_sub.obs['total_counts'].mean()

## Poisson GLM

First check if there are any datasets with only 1 sex or 1 smoking status, which would make the model overparameterized (not full rank).

In [41]:
np.any(adata_pseudo_tmp.obs.smoking_status.value_counts() == 1)

True

In [42]:
np.any(pd.crosstab(adata_pseudo_tmp.obs.smoking_status, adata_pseudo_tmp.obs.sex) == 1)

True

In [43]:
clusters

['Macrophages',
 'AT2',
 'Basal',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'AT1',
 'Submucosal Secretory',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Myofibroblasts',
 'MDC',
 'Airway smooth muscle']

In [44]:
# Poisson GLM loop
de_results_lvl3_glm = dict()

# Test over clusters
for clust in clusters:
    adata_pseudo_tmp = adata_pseudo_sub[adata_pseudo_sub.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_pseudo_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_pseudo_tmp, min_cells=4)
    if adata_pseudo_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if np.any(adata_pseudo_tmp.obs.sex.value_counts()==1):
        print(f'{clust} only has 1 male or 1 female sample.')
        continue
        
    print(f'Testing {adata_pseudo_tmp.n_vars} genes...')
    print(f'Testing in {adata_pseudo_tmp.n_obs} donors...')    
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_pseudo_tmp,
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_pseudo_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_pseudo_tmp.X[:, i],
            exog=dmat[0],
            offset=np.log(adata_pseudo_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Test over coefs
        for coef in tested_coef:
            de_results_temp = pois_results.wald_test(
                [x for i, x in enumerate(pois_model.exog_names) if dmat[1][i] in [coef]]
            )

            # Output the results nicely
            de_results_temp = pd.DataFrame({
                "gene": gene,
                "cell_identity": clust,
                "covariate": coef,
                "coef": pois_results.params[[y == coef for y in dmat[1]]],
                "coef_sd": pois_results.bse[[y == coef for y in dmat[1]]],                 
                "pval": de_results_temp.pvalue
            }, index= [clust+"_"+gene+"_"+coef])

            de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl3_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl3_glm = pd.concat([de_results_lvl3_glm[i] for i in de_results_lvl3_glm.keys()], ignore_index=True)

In cluster Macrophages:
male      64
female    46
Name: sex, dtype: int64
Testing 3 genes...
Testing in 110 donors...

In cluster AT2:
male      62
female    56
Name: sex, dtype: int64
Testing 3 genes...
Testing in 118 donors...

In cluster Basal:
male      62
female    48
Name: sex, dtype: int64
Testing 3 genes...
Testing in 110 donors...

In cluster T cell lineage:
male      65
female    53
Name: sex, dtype: int64
Testing 3 genes...
Testing in 118 donors...

In cluster Monocytes:
male      58
female    49
Name: sex, dtype: int64
Testing 3 genes...
Testing in 107 donors...

In cluster Multiciliated lineage:
male      91
female    79
Name: sex, dtype: int64
Testing 3 genes...
Testing in 170 donors...

In cluster AT1:
male      60
female    55
Name: sex, dtype: int64
Testing 3 genes...
Testing in 115 donors...

In cluster Submucosal Secretory:
male      25
female    20
Name: sex, dtype: int64
Testing 3 genes...
Testing in 45 donors...

In cluster Innate lymphoid cells:
male      40
fema

In [45]:
de_results_lvl3_glm.keys()

dict_keys(['Macrophages', 'AT2', 'Basal', 'T cell lineage', 'Monocytes', 'Multiciliated lineage', 'AT1', 'Submucosal Secretory', 'Innate lymphoid cells', 'Secretory', 'Capillary', 'Mast cells', 'B cell lineage', 'Fibroblasts', 'Dendritic cells', 'Venous', 'Lymphatic EC', 'Arterial', 'Rare', 'Myofibroblasts', 'MDC', 'Airway smooth muscle'])

In [46]:
full_res_lvl3_glm

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],-0.935355,8.554006,0.912927,0.994878
1,ACE2,Macrophages,age,0.001739,0.270826,0.994878,0.994878
2,TMPRSS2,Macrophages,sex[T.male],-0.172855,2.181250,0.936837,0.994878
3,TMPRSS2,Macrophages,age,-0.001233,0.073338,0.986589,0.994878
4,CTSL,Macrophages,sex[T.male],-0.122715,0.096598,0.203956,0.611867
5,CTSL,Macrophages,age,0.006942,0.003295,0.035154,0.210925
6,ACE2,AT2,sex[T.male],0.255630,1.871753,0.891369,0.941360
7,ACE2,AT2,age,0.004464,0.060061,0.940749,0.941360
8,TMPRSS2,AT2,sex[T.male],0.189991,0.221124,0.390227,0.941360
9,TMPRSS2,AT2,age,0.008379,0.006882,0.223424,0.941360


In [47]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],-0.935355,8.554006,0.912927,0.994878
1,ACE2,Macrophages,age,0.001739,0.270826,0.994878,0.994878
6,ACE2,AT2,sex[T.male],0.255630,1.871753,0.891369,0.941360
7,ACE2,AT2,age,0.004464,0.060061,0.940749,0.941360
12,ACE2,Basal,sex[T.male],0.163584,1.574816,0.917269,0.917269
13,ACE2,Basal,age,-0.014828,0.058307,0.799254,0.917269
18,ACE2,T cell lineage,sex[T.male],1.242744,7.481980,0.868080,0.990636
19,ACE2,T cell lineage,age,0.003331,0.283789,0.990636,0.990636
24,ACE2,Monocytes,sex[T.male],0.606111,8.219294,0.941215,0.941215
25,ACE2,Monocytes,age,-0.030483,0.350764,0.930747,0.941215


In [48]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
2,TMPRSS2,Macrophages,sex[T.male],-0.172855,2.181250,0.936837,0.994878
3,TMPRSS2,Macrophages,age,-0.001233,0.073338,0.986589,0.994878
8,TMPRSS2,AT2,sex[T.male],0.189991,0.221124,0.390227,0.941360
9,TMPRSS2,AT2,age,0.008379,0.006882,0.223424,0.941360
14,TMPRSS2,Basal,sex[T.male],0.379477,0.564662,0.501557,0.917269
15,TMPRSS2,Basal,age,0.011944,0.018977,0.529095,0.917269
20,TMPRSS2,T cell lineage,sex[T.male],0.323770,0.802562,0.686639,0.990636
21,TMPRSS2,T cell lineage,age,-0.014371,0.031465,0.647878,0.990636
26,TMPRSS2,Monocytes,sex[T.male],0.943658,1.345327,0.483032,0.941215
27,TMPRSS2,Monocytes,age,0.010799,0.047345,0.819576,0.941215


# Store results

In [53]:
full_res_lvl2_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl2_full.csv')

In [54]:
full_res_lvl3_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl3_full.csv')